In [ ]:
# inport BeautifulSoup
import re
from bs4 import BeautifulSoup

In [ ]:
folder = "20170513"
file=open(folder + "/a.html", "r")
soup = BeautifulSoup(file.read(), "html.parser")
file.close()

In [ ]:
#print(soup.find_all("div","playerInfo"))

In [ ]:
column = 'Name,Age,TSI,PayPerWeek,Form,Stamina,Special,Experience,Leadership,Loyalty,Keeper,Defending,Playmaking,Winger,Passing,Scoring,SetPieces'
f = open(folder + '/player.txt', 'a', encoding='utf-8')
f.write(column +'\n')
f.close()

In [ ]:
for div in soup.find_all("div","playerInfo"):
    # get String
    line = div.get_text()
    
    # parsing String
    line = line.replace(':\n                \n\n ', ':')
    line = line.replace('\n\n\n', '')
    line = line.replace('\n\n                        1', '\n1')
    line = line.replace('\n\n                        2', '\n2')
    line = line.replace('\n\n                        3', '\n3')
    line = line.replace('TSI = ','\nTSI:')
    line = line.replace('\xa0', '')
    line = line.replace(', 임금 = ', '\n임금:')
    line = line.replace('Won', ' Won\n')
    line = line.replace('form, ', 'form\n')
    if line.find('체력 [') > -1 :
        line = line.replace('체력 [', '체력\n특수능력:')
        line = line.replace("예측할 수 없음", "예측못함")
        line = line.replace(']', '\n')
    else :
        line = line.replace('체력', '체력\n특수능력:\n')
    line = line.replace('경험과 ', '경험\n')
    line = line.replace('리더십 능력', '리더십능력\n')
    line = line.replace('\n\n', '\n')
    line = line.replace('\n                    ', '')
    line = line.replace('충성도 ', '충성도:')
    line = line.replace('골키핑', '\n골키핑')
    line = line.replace('수비력','\n수비력')
    line = line.replace('플레이메이킹', '\n플레이메이킹')
    line = line.replace('윙어', '\n윙어')
    line = line.replace('패스력','\n패스력')
    line = line.replace('득점력', '\n득점력')
    line = line.replace('세트 피스','\n세트피스')
    
    line = line.replace('신의 경지','20')
    line = line.replace('초월적', '19')
    line = line.replace('마술적', '18')
    line = line.replace('전설적', '17')
    line = line.replace('외계인', '16')
    line = line.replace('막강함', '15')
    line = line.replace('초자연적', '14')
    line = line.replace('세계 최고', '13')
    line = line.replace('훌륭함', '12')
    line = line.replace('눈부심', '11')
    line = line.replace('걸출함', '10')
    line = line.replace('강력함', '9')
    line = line.replace('뛰어남', '8')
    line = line.replace('우수', '7')
    line = line.replace('무난함', '6')
    line = line.replace('불충분함', '5')
    line = line.replace('약함', '4')
    line = line.replace('나쁨', '3')
    line = line.replace('형편 없음', '2')
    line = line.replace('최악', '1')
    line = line.replace('없음', '0')
    
    line = line.replace('(\n', '(')
    line = line.replace('\n (', ':')
    line = line.replace(')', '')
    
    line = line.replace('Je-Min Jung    ', 'Je-Min Jung\n')
    line = line.replace('\n', ',')
    
    # name, age, TSI, pay per week, form, health(체력),
    
    line = line.replace(' 세', '세')
    line = line.replace(' 일', '일')
    line = line.replace('TSI:', '')
    line = line.replace('임금:주급 ', '')
    line = line.replace('0 Won','0')
    line = line.replace(' form', '')
    line = line.replace(' 체력', '')
    line = line.replace('특수능력:', '')
    line = line.replace(' 경험', '')
    line = line.replace(' 리더십능력', '')
    line = line.replace('충성도:', '')
    line = line.replace('골키핑:', '')
    line = line.replace('수비력:', '')
    line = line.replace('플레이메이킹:', '')
    line = line.replace('윙어:', '')
    line = line.replace('패스력:', '')
    line = line.replace('득점력:', '')
    line = line.replace('세트피스:', '')
    
    line = re.sub(r'[0-9]{4}-[0-9]{2}-[0-9]{2}:.*,', "", line)
    
    line.encode('utf-8')
    f = open(folder + '/player.txt', 'a', encoding='utf-8')
       
    for lines in line.split('\n'):
        if (re.match('^[0-9]{1,2}\. ', lines)) :
            lines = re.sub(r',$', "", lines)
            print(lines)
            f.write(lines +'\n')
    f.close()

In [1]:
import psycopg2

In [2]:
connect_str = "dbname='mydatabase' user='myuser' host='localhost' port='65432' password='123qwe'"
connect_str

"dbname='mydatabase' user='myuser' host='localhost' port='65432' password='123qwe'"

In [3]:
try:
    # use our connection values to establish a connection
    connect_str = "dbname='mydatabase' user='myuser' host='localhost' port='65432' password='123qwe'"
    conn = psycopg2.connect(connect_str)
    
except Exception as e:
    print("Uh oh, can't connect. Invalid dbname, user or password?")
    print(e)

In [4]:
# create a psycopg2 cursor that can execute queries
cursor = conn.cursor()
cursor

<cursor object at 0x7fc9a4d63140; closed: 0>

In [5]:
cursor.execute("""DROP TABLE IF EXISTS player """)

In [6]:
cursor.execute("""CREATE TABLE player (date VARCHAR(8)) """)

In [7]:
# insert a row to table
cursor.execute("""INSERT INTO player (date) VALUES ('20170609') ;""")

In [8]:
# run a SELECT statement - no data in there, but we can try it
cursor.execute("""SELECT * from player ;""")
rows = cursor.fetchall()
print(rows)

[('20170609',)]


In [9]:
conn.commit()

In [10]:
conn.close()

In [11]:
def selectAllPlayer(tableName):
    conn = None
    try:
        conn = psycopg2.connect(connect_str)
        cur = conn.cursor()
        cur.execute("SELECT * FROM " + tableName)
        row = cur.fetchone()
        while row is not None:
            print(row)
            row = cur.fetchone()
    except (Exception, psycopg2.DatabaseError) as error:
            print(error)
    finally:
        if conn is not None:
            conn.close()

In [12]:
selectAllPlayer("player")

('20170609',)
